In [1]:
#Importing needed libraries
import requests
import pandas as pd
from datetime import datetime
import geopandas as gpd
import fiona
import time
from json import JSONDecodeError
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.discrete.count_model import ZeroInflatedPoisson
from personal_lib import general_functions as gf


from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone_test")

C:\Users\johnf\AppData\Roaming\Python\Python310\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
### NYC ENERGY DATA BUILDINGS

In [ ]:
### Pulling in the Data Via the APi online 

##### Skip here until reading in files from CSV, because this takes very long to run.

In [211]:
## There are mulitple sources of yearly data. Compiling the source URLs here as well as the API urls to grab
## privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2
building_energy_LL84_sources={"2022+":{
    "api":"https://data.cityofnewyork.us/resource/5zyy-y8am.json",
    "info":"https://data.cityofnewyork.us/Environment/NYC-Building-Energy-and-Water-Data-Disclosure-for-/5zyy-y8am/about_data"
                              },
                              "2021":{
    "api":"https://data.cityofnewyork.us/resource/7x5e-2fxh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/7x5e-2fxh/about_data"
                              },
                              "2020":{
    "api":"https://data.cityofnewyork.us/resource/usc3-8zwd.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/usc3-8zwd/about_data"
                                  },
                              "2019":{
    "api":"https://data.cityofnewyork.us/resource/wcm8-aq5w.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/wcm8-aq5w/about_data"
                                  },
                              "2018":{
    "api":"https://data.cityofnewyork.us/resource/4tys-3tzj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4tys-3tzj/about_data"
                                  },
                              "2017":{
    "api":"https://data.cityofnewyork.us/resource/4t62-jm4m.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4t62-jm4m/about_data"
                                  },
                              "2016":{
    "api":"https://data.cityofnewyork.us/resource/utpj-74fz.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/utpj-74fz/about_data"
                                  },
                              "2015":{
    "api":"https://data.cityofnewyork.us/resource/77q4-nkfh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/77q4-nkfh/about_data"
                                  },
                              "2014":{
    "api":"https://data.cityofnewyork.us/resource/nbun-wekj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/nbun-wekj/about_data"
                                  },
                              "2013":{
    "api":"https://data.cityofnewyork.us/resource/yr5p-wjer.json",
    "info":"http://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/yr5p-wjer"
                                  },
                              "2012":{
    "api":"https://data.cityofnewyork.us/resource/r6ub-zhff.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/r6ub-zhff/about_data"
                                  },
                              "2011":{
    "api":"https://data.cityofnewyork.us/resource/k7nh-aufb.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/k7nh-aufb/about_data"
                                  },
                              "2010":{
    "api":"https://data.cityofnewyork.us/resource/kswi-37bp.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/kswi-37bp/about_data"
                                  },
#Monthly Data? 2018-2023
                              "Monthly":{
    "api":"https://data.cityofnewyork.us/resource/fvp3-gcb2.json",
    "info":"https://data.cityofnewyork.us/Environment/Local-Law-84-Monthly-Data/fvp3-gcb2/about_data"
    }}


In [ ]:
PAGE = 1000               # <-- enforce 1,000 rows per page
TIMEOUT = 30
MAX_RETRIES = 5
BACKOFF_BASE = 1.5

session = requests.Session()
headers = {}

def fetch_all_rows_1k(api_url: str, source_years: str, source_info_url: str) -> pd.DataFrame:
    offset = 0
    frames = []

    while True:
        params = {"$limit": PAGE, "$offset": offset}

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = session.get(api_url, params=params, headers=headers, timeout=TIMEOUT)
                if resp.status_code in (429, 502, 503, 504):
                    time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))
                    continue
                resp.raise_for_status()

                if "json" not in resp.headers.get("Content-Type", "").lower():
                    preview = resp.text[:200]
                    raise ValueError(f"Non-JSON response (status {resp.status_code}): {preview}")

                data_chunk = resp.json()
                if not data_chunk:
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

                # Create DataFrame
                df = pd.DataFrame(data_chunk)

                # 🔑 Add your metadata columns here
                df["source_years"] = source_years
                df["source_api_url"] = api_url
                df["source_info_url"] = source_info_url

                frames.append(df)

                # If less than PAGE, stop; otherwise keep paginating
                if len(data_chunk) < PAGE:
                    return pd.concat(frames, ignore_index=True)

                offset += PAGE
                break  # Success, go to next page

            except (requests.RequestException, JSONDecodeError, ValueError) as e:
                if attempt == MAX_RETRIES:
                    print(f"⚠️ Failed fetching {api_url} at offset {offset}: {e}")
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
                time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))


In [ ]:
agg_running_list = []
for k, v in building_energy_LL84_sources.items():
    print(f"Fetching {k} -> {v['api']}")
    df = fetch_all_rows_1k(v["api"], k, v["info"])
    if not df.empty:
        agg_running_list.append(df)
    else:
        print(f"Warning: no rows returned for {k} ({v['api']}).")

nyc_building_energy = pd.concat(agg_running_list, ignore_index=True) if agg_running_list else pd.DataFrame()
print(f"Total rows: {len(nyc_building_energy)}")

#### End Skip here. Pick up and read in data

In [213]:
### If just want residential limist continue to skip in untill reading in residential 3
# nyc_building_energy=  pd.read_csv("nyc_dob_energy_2010_2024.csv")

In [ ]:
## Formatting into proper df
# print(nyc_building_energy.shape)
## Saving to File
# nyc_building_energy.to_csv("nyc_dob_energy_2010_2024.csv",index=False)

### Starting Cleaning / Skip if you want ot just read in residential3

In [ ]:
### Columns to Drop because potentially irrelevant or redundant for 
nyc_building_energy_processing = nyc_building_energy.copy()
## Dropping Other Property Type Columns 
to_drop =[
## Banking oriented COlumns
'bank_branch_computer_density',
 'bank_branch_gross_floor_area',
 'bank_branch_gross_floor_area_ft',
 'bank_branch_number_of',
 'bank_branch_number_of_workers',
 'bank_branch_percent_that',
 'bank_branch_weekly_operating',
 'bank_branch_worker_density',
## College / Uni / School
'college_university_gross',
 'college_university_gross_floor_area_ft',
 'college_university_number',
    'k_12_school_computer_density',
 'k_12_school_cooking_facilities',
 'k_12_school_gross_floor_area',
 'k_12_school_gross_floor_area_ft',
 'k_12_school_high_school',
 'k_12_school_percent_that',
 'k_12_school_refrigeration',
 'k_12_school_weekend_operation',
 'laboratory_gross_floor_area_ft',
    'library_gross_floor_area',
# OTher
"automobile_dealership_gross",
'convenience_store_without',
'data_center_energy_estimates_applied',
 'data_center_gross_floor_area',
 'data_center_gross_floor_area_ft',
 'data_center_it_energy',
 'data_center_it_energy_configuration',
 'data_center_it_equipment_input_meter_kwh',
 'data_center_it_site_energy',
 'data_center_it_site_energy_kwh',
 'data_center_it_source_energy_kbtu',
 'data_center_national_median',
 'data_center_pdu_input_meter_kwh',
 'data_center_pdu_output_meter_kwh',
 'data_center_ups_output_meter_kwh',
 'enclosed_mall_gross_floor',
 'enclosed_mall_gross_floor_area_ft',
 'fast_food_restaurant_gross',
 'financial_office_gross_floor',
 'financial_office_gross_floor_area_ft',
 'financial_office_number_of',
 'financial_office_number_of_1',
 'financial_office_number_of_computers',
 'financial_office_number_of_workers_on_main_shift',
 'financial_office_weekly',
 'financial_office_weekly_operating_hours',
 'fitness_center_health_club',
 'fitness_center_health_club_gym_gross_floor_area_ft',
 'food_sales_gross_floor_area',
 'food_sales_gross_floor_area_ft',
 'food_service_gross_floor',
 'food_service_gross_floor_area_ft',
     'worship_facility_computer',
 'worship_facility_cooking',
 'worship_facility_gross_floor',
 'worship_facility_gross_floor_area_ft',
 'worship_facility_weekly',
    'supermarket_grocery_cooking',
 'supermarket_grocery_gross',
 'supermarket_grocery_gross_floor_area_ft',
 'supermarket_grocery_number',
 'supermarket_grocery_number_1',
 'supermarket_grocery_number_2',
 'supermarket_grocery_number_of_open_or_closed_refrigeration_freezer_units',
 'supermarket_grocery_number_of_walk_in_refrigeration_freezer_units',
 'supermarket_grocery_percent',
 'supermarket_grocery_walk',
 'supermarket_grocery_weekly',
 'supermarket_grocery_worker',
 'swimming_pool_approximate',
 'swimming_pool_location_of',
 'swimming_pool_months_in_use',
     'social_meeting_hall_gross',
 'social_meeting_hall_gross_floor_area_ft',
     'residence_hall_dormitory',
 'residence_hall_dormitory_1',
 'residence_hall_dormitory_2',
 'residence_hall_dormitory_3',
 'residence_hall_dormitory_4',
 'residence_hall_dormitory_gross_floor_area_ft',
 'restaurant_gross_floor_area',
 'restaurant_gross_floor_area_ft',
 'restaurant_weekly_operating',
 'restaurant_weekly_operating_hours',
 'restaurant_worker_density',
 'restaurant_worker_density_number_per_1_000_sq_ft',
 'retail_store_cash_register',
 'retail_store_computer_density',
 'retail_store_exterior_entrance',
 'retail_store_gross_floor',
 'retail_store_gross_floor_area_ft',
 'retail_store_number_of_open',
 'retail_store_number_of_open_or_closed_refrigeration_freezer_units',
 'retail_store_number_of_walk',
 'retail_store_number_of_walk_in_refrigeration_freezer_units',
 'retail_store_open_or_closed',
 'retail_store_percent_that',
 'retail_store_walk_in',
 'retail_store_weekly_operating',
 'retail_store_worker_density',
 'self_storage_facility_gross',
 'self_storage_facility_gross_floor_area_ft',
 'senior_care_community_average',
 'senior_care_community_gross',
 'senior_care_community_maximum',
 'senior_care_community_number',
 'senior_care_community_number_1',
 'senior_care_community_number_2',
 'senior_care_community_number_3',
 'senior_care_community_number_4',
 'senior_care_community_number_5',
 'senior_care_community_number_6',
 'senior_care_community_percent',
 'senior_living_community_gross_floor_area_ft',
 'senior_living_community_living_unit_density_number_per_1_000_sq_ft',
    'movie_theater_gross_floor_area_ft',
    'non_refrigerated_warehouse',
 'non_refrigerated_warehouse_1',
 'non_refrigerated_warehouse_2',
 'non_refrigerated_warehouse_3',
 'non_refrigerated_warehouse_4',
 'non_refrigerated_warehouse_5',
 'non_refrigerated_warehouse_gross_floor_area_ft',
     'parking_completely_enclosed',
 'parking_completely_enclosed_parking_garage_size_ft',
 'parking_gross_floor_area',
 'parking_gross_floor_area_ft',
 'parking_open_parking_lot',
 'parking_open_parking_lot_size_ft',
 'parking_partially_enclosed',
 'parking_partially_enclosed_parking_garage_size_ft',
     'strip_mall_gross_floor_area',
## Hostpial / Hotel
    'hospital_general_medical',
 'hospital_general_medical_1',
 'hospital_general_medical_10',
 'hospital_general_medical_11',
 'hospital_general_medical_12',
 'hospital_general_medical_13',
 'hospital_general_medical_14',
 'hospital_general_medical_15',
 'hospital_general_medical_16',
 'hospital_general_medical_17',
 'hospital_general_medical_2',
 'hospital_general_medical_3',
 'hospital_general_medical_4',
 'hospital_general_medical_5',
 'hospital_general_medical_6',
 'hospital_general_medical_7',
 'hospital_general_medical_8',
 'hospital_general_medical_9',
 'hotel_amount_of_laundry',
 'hotel_cooking_facilities',
 'hotel_full_service_spa_floor',
 'hotel_gross_floor_area_ft',
 'hotel_gym_fitness_center',
 'hotel_gym_fitness_center_floor_area_ft',
 'hotel_number_of_rooms',
 'hotel_percent_that_can_be',
 'hotel_room_density_number',
 'hotel_type_of_laundry_facility',
 'hotel_worker_density_number',
 'urgent_care_clinic_other',
 'urgent_care_clinic_other_outpatient_gross_floor_area_ft',
    'mailing_center_post_office_gross_floor_area_ft',
 'manufacturing_industrial_plant_gross_floor_area_ft',
 'medical_office_gross_floor',
 'medical_office_gross_floor_area_ft',
 'medical_office_mri_machine',
 'medical_office_number_of',
 'medical_office_number_of_1',
 'medical_office_number_of_computers',
 'medical_office_number_of_mri_machines',
 'medical_office_number_of_workers_on_main_shift',
 'medical_office_percent_that',
 'medical_office_percent_that_1',
 'medical_office_percent_that_can_be_cooled',
 'medical_office_percent_that_can_be_heated',
 'medical_office_weekly',
 'medical_office_weekly_operating_hours',
    'museum_gross_floor_area_ft',
    'office_computer_density_number',
 'office_gross_floor_area_ft',
 'office_number_of_computers',
 'office_number_of_workers',
 'office_number_of_workers_on_main_shift',
 'office_percent_that_can_be',
 'office_percent_that_can_be_1',
 'office_percent_that_can_be_cooled',
 'office_percent_that_can_be_heated',
 'office_weekly_operating_hours',
 'office_worker_density_number',
 'office_worker_density_number_per_1_000_sq_ft',
     'adult_education_gross_floor',
 'adult_education_gross_floor_area_ft',
    'data_center_ups_output_meter',
 'data_center_pdu_input_meter',
 'data_center_pdu_output_meter',
 'data_center_it_equipment',
 'data_center_it_site_energy',
 'data_center_it_source_energy',
 'data_center_pue',
 'data_center_national_median',
 'data_center_gross_floor_area',
 'data_center_ups_system',
 'data_center_it_energy',
 'data_center_cooling_equipment',
'supermarkets_grocery_gross',
 'supermarkets_grocery_number',
 'supermarkets_grocery_number_1',
 'supermarkets_grocery_number_2',
 'supermarkets_grocery_percent',
 'supermarkets_grocery_presence',
 'supermarkets_grocery_walk',
 'supermarkets_grocery_weekly',
 'supermarkets_grocery_workers',
'house_of_worship_gross_floor',
 'house_of_worship_pc_density',
 'house_of_worship_weekly',
 'house_of_worship_presence',
'residence_halls_dormitories',
'medical_office_percent_cooled',
 'residence_halls_dormitories_1',
 'residence_halls_dormitories_2',
 'residence_halls_dormitories_3',
 'residence_halls_dormitories_4',
'hotel_onsite_laundry_short',
 'warehouse_unrefrigerated',
 'warehouse_unrefrigerated_1',
 'warehouse_unrefrigerated_2',
 'warehouse_unrefrigerated_3',
 'warehouse_unrefrigerated_4',
 'warehouse_unrefrigerated_5',
 'warehouse_unrefrigerated_6',
 'hospital_gross_floor_area',
 'hospital_laboratory_y_1_n',
 'hospital_laundry_facility',
 'hospital_maximum_number_of',
 'hospital_number_of_buildings',
 'warehouse_refrigerated_gross',
 'warehouse_refrigerated_weekly',
 'warehouse_refrigerated_workers',
 'hospital_number_of_licensed',
 'multifamily_home_dishwashers']
nyc_building_energy_processing = nyc_building_energy_processing.drop(columns=to_drop)
## Fruther Limiting to those that are under 10 Stories. 
nyc_building_energy_processing = nyc_building_energy_processing.replace("Not Available",np.nan).dropna(how='all',axis=1)
nyc_building_energy_processing = nyc_building_energy_processing.dropna(how='all',axis=1)

nyc_building_energy_processing["primary_property_type"] = nyc_building_energy_processing["primary_property_type"].combine_first(nyc_building_energy_processing["primary_property_type_epa"])
nyc_building_energy_processing = nyc_building_energy_processing.drop(columns=["primary_property_type_epa"])
nyc_building_energy_processing["primary_property_type_self"] = nyc_building_energy_processing["primary_property_type_self"].combine_first(nyc_building_energy_processing["primary_property_type_self_selected"])
nyc_building_energy_processing = nyc_building_energy_processing.drop(columns=["primary_property_type_self_selected"])

In [ ]:
## Beginning of Limiting to Multifamily Homes (Step 1)
residential = nyc_building_energy_processing[(
    (nyc_building_energy_processing["list_of_all_property_use"].isin([i for i in nyc_building_energy_processing["list_of_all_property_use"].unique() if 'Multifamily Housing' in str(i)]))
    |(nyc_building_energy_processing["list_of_all_property_use"].isnull() 
      & (nyc_building_energy_processing["primary_property_type_self"]=="Multifamily Housing")
      & ((nyc_building_energy_processing["primary_property_type"]=="Multifamily Housing")|
         (nyc_building_energy_processing["primary_property_type"].isnull()))))]

In [ ]:
## Beginning of Limiting to Multifamily Homes (Step 1)
residential = nyc_building_energy_processing[(
    (nyc_building_energy_processing["list_of_all_property_use"].isin([i for i in nyc_building_energy_processing["list_of_all_property_use"].unique() if 'Multifamily Housing' in str(i)]))
    |(nyc_building_energy_processing["list_of_all_property_use"].isnull() 
      & (nyc_building_energy_processing["primary_property_type_self"]=="Multifamily Housing")
      & ((nyc_building_energy_processing["primary_property_type"]=="Multifamily Housing")|
         (nyc_building_energy_processing["primary_property_type"].isnull()))))]

In [ ]:
print(residential.shape)

In [ ]:
## Second Step is limiting to NON MIXED USE, so ONLY MultiFamily Residences
residential = residential[
    residential['list_of_all_property_use'].isnull() |
    residential['list_of_all_property_use'].apply(
        lambda x: len(x.split(",")) == 1 if isinstance(x, str) else False
    )
]
## Fruther Limiting to those that are under 10 Stories. 
residential = residential.replace("Not Available",np.nan).dropna(how='all',axis=1)
residential2 = residential.dropna(how='all',axis=1)

In [ ]:
# Convert year_ending to datetime
residential2["year_ending"] = pd.to_datetime(residential2["year_ending"], errors="coerce")
# Optional: extract just the year if that’s all you need
residential2["year_ending_year"] = residential2["year_ending"].dt.year
residential2[["report_year","year_ending_year","source_years","source_api_url","source_info_url"]][residential['report_year'].isnull()]

In [ ]:
## Limtiing to the Metered Areas for whole property or whole building
residential3 = residential2[residential2["metered_areas_energy"].isin(['Whole Building', "Whole Property"])]
residential3 = residential3[residential3["primary_property_type_self"]=='Multifamily Housing'] ## confirming onlyl multifam. No nulls here

residential3 = residential3[(residential3["construction_status"]=='Existing')|(residential3["construction_status"].isnull())]
## Stand Alone Properties
residential3 = residential3[(residential3["parent_property_id"].isnull())|(residential3["parent_property_id"]=='Not Applicable: Standalone Property')]
residential3 = residential3.drop(columns=["primary_property_type_self",
                                          "primary_property_type",
                                          "national_median_reference",
                                          "list_of_all_property_use","largest_property_use_type","construction_status",
                                         "parent_property_id","parent_property_name"])

##### End second Skip

In [184]:
residential3 = pd.read_csv("residential3.csv") 

C:\Users\johnf\AppData\Local\Temp\ipykernel_36636\3537755669.py:1: DtypeWarning: Columns (4,5,8,14,18,19,20,43,44,45,46,47,53,54,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,98,112,118,126,127,128,129,130,131,132,133,138,139,146,147,148,151,154,155,156,160,163,172,173,175,176,177,178,179,180,181,183,185,188,189,190,210,211,231,245,247,248,249,250,251,252,253,254,255,256,257,258,259,260,272,273,274,275,276,277,281,282,286,289,295,296,303,304,305,306,307,308,309,318,319,333,334,335,336,337,339,342,345,347,348,352) have mixed types. Specify dtype option on import or set low_memory=False.
  residential3 = pd.read_csv("residential3.csv")


#### Limint by Years and for those Buildigns that have been i nthe Data for the years we want

In [185]:
buildings_2012_2013_ids = residential3[residential3["year_ending_year"].isin([2012,2013])]["property_id"].unique()
print("CheckingGrouping")
print(residential3[residential3['property_id'].isin(buildings_2012_2013_ids)].groupby(['year_ending_year']).agg({"year_ending":"count"}).reset_index())
### Limtiing to 2017 and before 
residential3_2012_2017 = residential3[residential3["year_ending_year"]<2018]
print("YEARS")
print(residential3_2012_2017['year_ending_year'].unique())
residential3_2012_2017 = residential3_2012_2017[residential3_2012_2017['property_id'].isin(buildings_2012_2013_ids)]

CheckingGrouping
    year_ending_year  year_ending
0               2012           32
1               2013         9275
2               2014         7475
3               2015         4429
4               2016         3908
5               2017         5104
6               2018         3365
7               2019         3199
8               2020         4399
9               2021         2547
10              2022         2644
11              2023         2773
YEARS
[2017 2016 2015 2014 2013 2012]


In [217]:
### Building Info Only 
building_info =residential3_2012_2017[["property_id","address_1","address_2","city","postal_code",'county','borough',"latitude","longitude"]].drop_duplicates()
# building_info.to_csv("building_info.csv", index=False)

In [76]:
building_info = pd.read_csv("building_info.csv")

In [218]:
# --- PATCHED HELPERS (extend yours) ---
import re
import pandas as pd
from typing import Optional, List, Tuple

STATE_ABBR = "NY"

# NYC biasing bounds (rough NYC box)
NYC_BOUNDS = {
    "southwest": {"lat": 40.477399, "lng": -74.259090},
    "northeast": {"lat": 40.917577, "lng": -73.700272},
}

# Common misspellings / normalizations seen in your data
COMMON_FIXES = {
    r"\bWASHIGNTON\b": "WASHINGTON",
    r"\bWASHNGTN\b": "WASHINGTON",
    r"\bALBERMALE\b": "ALBEMARLE",
    r"\bALBERMALE\b": "ALBEMARLE",
    r"\bAMSETRDAM\b": "AMSTERDAM",
    r"\bKINGBRIDGE\b": "KINGSBRIDGE",
    r"\bOVINGTON\b": "OVINGTON",   # keep as is (looks fine)
    r"\bNAGLE HOUSE\b": "NAGLE AVE",  # likely street not building name
    r"\bNAGLE\b": "NAGLE",  # pass-through
    r"\bST\s*NICOLAS\b": "ST NICHOLAS",
    r"\bOILVER\b": "OLIVER",
    r"\bRIVER SIDE\b": "RIVERSIDE",
    r"\bSTEET\b": "STREET",
    r"\bSREET\b": "STREET",
    r"\bFOR WASHINGTON\b": "FORT WASHINGTON",
    r"\bFT\.?\b": "FORT",
    r"\bBIVONA\b": "BIVONA",  # leave
    r"\bCLARKE\b": "CLARKE",  # leave
    r"\bCAROLL\b": "CARROLL",
    r"\bCLAFFIN\b": "CLAFLIN",
    r"\bDEKALB\b": "DEKALB",
    r"\bLAFAYETTE\b": "LAFAYETTE",
    r"\bW\s*MOSHOLU\b": "W MOSHOLU",
    r"\bE\s*MOSHOLU\b": "E MOSHOLU",
    r"\bMOSHOLU\b": "MOSHOLU",
    r"\bPARSON\b": "PARSONS",
    r"\bPARSONS BLVD?\b": "PARSONS BLVD",
}

# Street type vocabulary to help "best-guess" if clearly missing
STREET_TYPES = ["ST", "AVE", "RD", "BLVD", "PL", "LN", "DR", "CT", "PKWY", "TER", "PLZ"]

def _norm_ws(s: Optional[str]) -> Optional[str]:
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return None
    s = str(s)
    # HTML escapes commonly seen
    s = s.replace("&amp;", "&")
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s if s else None

def _apply_common_fixes(u: str) -> str:
    for pat, repl in COMMON_FIXES.items():
        u = re.sub(pat, repl, u, flags=re.IGNORECASE)
    return u

def _std_unit(s: Optional[str]) -> Optional[str]:
    u = _norm_ws(s)
    if not u:
        return None
    u = u.upper().replace("#", "").strip()
    if not u:
        return None
    u = re.sub(r"^(APARTMENT|APT\.?)\s*", "APT ", u)
    u = re.sub(r"^(SUITE|STE\.?)\s*", "STE ", u)
    u = re.sub(r"^(FLOOR|FL\.?)\s*", "FL ", u)
    if not re.match(r"^(APT|STE|FL)\b", u):  # bare "5B" → "APT 5B"
        u = f"APT {u}"
    return u

def _std_street(u: Optional[str]) -> Optional[str]:
    u = _norm_ws(u)
    if not u:
        return None
    u = _apply_common_fixes(u.upper())

    repl = {
        " STREET": " ST",
        " AVENUE": " AVE",
        " ROAD": " RD",
        " BOULEVARD": " BLVD",
        " PLACE": " PL",
        " LANE": " LN",
        " DRIVE": " DR",
        " COURT": " CT",
        " PARKWAY": " PKWY",
        " TERRACE": " TER",
        " PLAZA": " PLZ",
    }
    for k, v in repl.items():
        u = re.sub(k + r"\b", v, u)

    # Normalize ordinals and add missing "TH/ST/ND/RD" on bare numbers like "E 48" or "West104"
    u = re.sub(r"\b(\d+)\s*(ST|ND|RD|TH)\b", lambda m: f"{int(m.group(1))}{m.group(2)}", u)
    u = re.sub(r"\b(WEST|W|EAST|E)\s*(\d{1,3})(?=\b)", r"\1 \2", u)  # ensure space
    def add_ordinal(m):
        n = int(m.group(2))
        suf = "TH"
        if n % 10 == 1 and n % 100 != 11: suf = "ST"
        elif n % 10 == 2 and n % 100 != 12: suf = "ND"
        elif n % 10 == 3 and n % 100 != 13: suf = "RD"
        return f"{m.group(1)} {n}{suf}"
    u = re.sub(r"\b(WEST|W|EAST|E)\s+(\d{1,3})\b(?!\s*(ST|AVE|RD|BLVD|PL|LN|DR|CT|PKWY|TER|PLZ))", add_ordinal, u)

    # Collapse multiple addresses to last one here (we also split earlier in the pipeline)
    if "/" in u:
        u = u.split("/")[-1].strip()

    # Queens: hyphenated house numbers like "41-07 42ND ST" are valid; leave them as-is
    # Best-guess add a street type if line is like "308 WEST 104TH" (no type) or "160 E 48TH"
    if re.search(r"\b(WEST|W|EAST|E|NORTH|N|SOUTH|S)\b", u) and re.search(r"\b\d{1,3}(ST|ND|RD|TH)\b", u) and not re.search(r"\b(ST|AVE|RD|BLVD|PL|LN|DR|CT|PKWY|TER|PLZ)\b", u):
        u = u + " ST"

    # Convert key phrases
    u = re.sub(r"\bFT\b", "FORT", u)

    return u.title()

def _borough_from_fields(borough, county, city) -> Optional[str]:
    vals = " ".join([str(x) for x in [borough, county, city] if pd.notna(x)]).upper()
    if any(x in vals for x in ["MANHATTAN", "NEW YORK", "NY COUNTY"]): return "MANHATTAN"
    if "BRONX" in vals: return "BRONX"
    if any(x in vals for x in ["BROOKLYN", "KINGS"]): return "BROOKLYN"
    if "QUEENS" in vals: return "QUEENS"
    if any(x in vals for x in ["STATEN", "RICHMOND"]): return "STATEN ISLAND"
    return None

def _usps_city_from_borough(city, borough_norm) -> Optional[str]:
    c = (_norm_ws(city) or "").upper()
    if borough_norm == "MANHATTAN": return "New York"
    if borough_norm == "BRONX": return "Bronx"
    if borough_norm == "BROOKLYN": return "Brooklyn"
    if borough_norm == "QUEENS":
        # USPS accepts neighborhoods, but "Queens" is safest default
        return "Queens" if c in ["", "QUEENS", "NAN"] else c.title()
    if borough_norm == "STATEN ISLAND": return "Staten Island"
    return c.title() if c else None

def _std_zip5(z):
    if z is None or (isinstance(z, float) and pd.isna(z)):
        return None
    s = re.sub(r"\D", "", str(z))
    return s[:5] if len(s) >= 5 else None

# ---------- NEW: address candidate generator ----------

SEP_PATTERN = re.compile(r"\s*(?:;|,|/|&{1,2}| and | AND |\bETAL\b|\bet al\b|\(|\)|\bAKA\b|\bA\/K\/A\b)\s*", flags=re.IGNORECASE)

def _is_pobox(s: str) -> bool:
    return bool(re.search(r"\bP\.?\s*O\.?\s*BOX\b", s, flags=re.IGNORECASE))

def _has_street_name(u: str) -> bool:
    # needs something besides just a house number or zeros
    return bool(re.search(r"\b[A-Z][A-Z]+\b", u)) and not re.fullmatch(r"\d+(-\d+)?", re.sub(r"[^0-9\-]", "", u or ""))

def _expand_number_ranges(u: str) -> List[str]:
    """
    '2078-84-90 Morris Ave' -> ['2078 Morris Ave','2084 Morris Ave','2090 Morris Ave']
    '1051-1057-1061-1065 Boston Rd' -> same expansion
    """
    m = re.search(r"\b(\d{1,6}(?:-\d{1,6})+)\s+([A-Z].+)$", u)
    if not m:
        return [u]
    block = m.group(1)
    tail = m.group(2)
    parts = [p for p in re.split(r"-", block) if p]
    base = parts[0]
    expanded = []
    for p in parts:
        if len(p) < len(base):  # "2078-84" -> fill high-order digits from base
            p = base[:len(base)-len(p)] + p
        expanded.append(f"{p} {tail}")
    return expanded

def _split_multi(s: str) -> List[str]:
    s = SEP_PATTERN.sub(" | ", s)  # unify to pipe, then split
    tokens = [t.strip(" .") for t in s.split("|") if _norm_ws(t)]
    # Remove address-like fragments that are obviously units/notes only
    return [t for t in tokens if t and not re.match(r"^(APT|STE|FL|B#\d+)\b", t, flags=re.IGNORECASE)]

def build_candidates(row) -> List[Tuple[str, dict, str]]:
    """
    Returns list of (query, components, reason_tag)
    """
    a1 = _std_street(row.get("address_line1_clean"))
    a2 = _std_unit(row.get("address_line2_clean"))
    borough = _borough_from_fields(row.get("borough_clean"), row.get("county"), row.get("city_clean"))
    city = _usps_city_from_borough(row.get("city_clean"), borough)
    zip5 = _std_zip5(row.get("zip"))
    state = STATE_ABBR

    raw = _norm_ws(row.get("geocode_key") or row.get("address_1") or "")
    raw = _apply_common_fixes(raw.upper()) if raw else ""
    raw_parts = _split_multi(raw) if raw else []

    # Build an address seed from clean fields
    seeds = []
    if a1:
        seeds.append(a1)
    seeds.extend(_expand_number_ranges(x) for x in raw_parts)
    seeds = [y for x in seeds for y in (x if isinstance(x, list) else [x])]
    seeds = [s for s in seeds if s] or ([raw.title()] if raw else [])

    candidates = []
    for s in seeds:
        s_norm = _std_street(s)
        if not s_norm:
            continue
        if _is_pobox(s_norm):
            candidates.append((None, {}, "POBOX"))
            continue
        if not _has_street_name(s_norm):
            candidates.append((None, {}, "MISSING_STREET"))
            continue

        line = s_norm
        parts = [line]
        # Append unit if it looks like an apartment (rarely helpful for geocode but OK)
        # if a2: parts.append(a2)

        locality = city or (borough.title() if borough else None)
        tail = ", ".join([p for p in [locality, state, zip5] if p])
        query = f"{' '.join(parts)}, {tail}" if tail else " ".join(parts)

        comps = {"administrative_area": state}
        if locality: comps["locality"] = locality

        candidates.append((query, comps, "OK"))
    # Deduplicate, keep order
    seen = set()
    out = []
    for q, c, r in candidates:
        key = (q or r, tuple(sorted(c.items())))
        if key not in seen:
            seen.add(key)
            out.append((q, c, r))
    return out


In [219]:
# Cleaned drafts
building_info["address_line1_clean"] = building_info["address_1"].apply(_std_street)
building_info["address_line2_clean"] = building_info["address_2"].apply(_std_unit)

building_info["borough_clean"] = [
    _borough_from_fields(b, c, ci)
    for b, c, ci in zip(building_info.get("borough"), building_info.get("county"), building_info.get("city"))
]
building_info["city_clean"] = [
    _usps_city_from_borough(ci, bn)
    for ci, bn in zip(building_info.get("city"), building_info["borough_clean"])
]
building_info["state_clean"] = STATE_ABBR
building_info["postal_code_5_clean"] = building_info.get("postal_code").apply(_std_zip5) if "postal_code" in building_info.columns else None

# Mark rows missing coords
building_info["needs_geocoding"] = building_info["latitude"].isna() | building_info["longitude"].isna()

# a stable address key to dedupe geocoding calls
def addr_key(r):
    parts = [
        _norm_ws(r["address_line1_clean"]) or "",
        _norm_ws(r["address_line2_clean"]) or "",
        _norm_ws(r["city_clean"]) or "",
        STATE_ABBR,
        _norm_ws(r["postal_code_5_clean"]) or "",
    ]
    return "|".join(parts).upper()

building_info["geocode_key"] = building_info.apply(addr_key, axis=1)

building_info.head(3)[[
    "property_id","address_1","address_2","city","borough","county",
    "address_line1_clean","address_line2_clean","city_clean","state_clean","postal_code_5_clean",
    "needs_geocoding","geocode_key"
]]


,property_id,address_1,address_2,city,borough,county,address_line1_clean,address_line2_clean,city_clean,state_clean,postal_code_5_clean,needs_geocoding,geocode_key
85955,2638790.0,43-22,NaN,Sunnyside,NaN,NaN,43-22,None,Sunnyside,NY,None,True,43-22||SUNNYSIDE|NY|
85957,2707907.0,2626 Homecrest Avenue,NaN,Brooklyn,BROOKLYN,NaN,2626 Homecrest Ave,None,Brooklyn,NY,None,False,2626 HOMECREST AVE||BROOKLYN|NY|
85974,3521602.0,3240 Henry Hudson parkway,NaN,Bronx,BRONX,NaN,3240 Henry Hudson Pkwy,None,Bronx,NY,None,False,3240 HENRY HUDSON PKWY||BRONX|NY|


In [220]:
print(building_info.shape)
building_info = building_info.dropna(subset=["address_1","address_2","city","postal_code","county","borough",
                                             "latitude","longitude","address_line1_clean"],how='all')
print(building_info.shape)

(9739, 17)
(9720, 17)


In [221]:
### Geocode The Null lat long 
with_coord = building_info[building_info["needs_geocoding"]==False]
print(with_coord.shape)
without_coord = building_info[building_info["needs_geocoding"]==True]
print(without_coord.shape)

(9121, 17)
(599, 17)


In [222]:
without_coord['city'][without_coord['city']=='Flushinig']='Flushing'
without_coord['borough_clean'][(without_coord['city'].isin(['Sunnyside', 'Astoria', 'Jackson Heights', 'Flushing', 'Rego Park',
                                                   'Forest Hills', 'Oakland Gardens', 'Bayside', 'Kew Gardens','flushing',
                                                   'Woodside', 'Flushinig', 'Larchmont', 'Ridgewood','Jamaica', 'Albertson'])
                                &(without_coord['borough_clean'].isnull()))]='QUEENS'
### Manual Fixes where possible 
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="3115 brighton 6th"))]="BROOKLYN"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="144-35/39 Sanford Avenue"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="183-11 Hillside Ave"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="103-30/26 68th Ave"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="71-11 -71-23 162ND STREET"))]="QUEENS"
without_coord = without_coord[~without_coord['borough_clean'].isnull()]
print(without_coord.shape)

(586, 17)


C:\Users\johnf\AppData\Local\Temp\ipykernel_42220\277626894.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  without_coord['city'][without_coord['city']=='Flushinig']='Flushing'
C:\Users\johnf\AppData\Local\Temp\ipykernel_42220\277626894.

In [223]:
print(without_coord.shape)

(586, 17)


### Start of Geocoding 

In [2]:
creds = gf.get_creds()
google_api_key = creds["Google"]["geocoding"]
open_ai = creds["openai"]["FirstTestKey"]

In [ ]:
# reading in past resu;ts
# results_df = pd.concat(results)
# results_df.to_csv("results_geo.csv",index=False)
# results_df = pd.read_csv("results_geo.csv")
# failed_df = pd.concat(failed)
# failed_df.to_csv("failed_geo.csv",index=False)
# failed_df = pd.read_csv("failed_geo.csv")

In [207]:
results = []
failed = []

In [228]:
# ---- GEOCODING LOOP (with retries/candidates/NYC bias) ----
import googlemaps
import os
import math
gmaps = googlemaps.Client(key=google_api_key)

results, failed = [], []

def geocode_row(row):
    cands = build_candidates(row)
    # Always add *raw borough + state* fallback for named complexes (e.g., NYCHA) to try to at least anchor
    borough = _borough_from_fields(row.get("borough_clean"), row.get("county"), row.get("city_clean"))
    if borough:
        cands.append((f"{borough.title()}, NY", {"administrative_area": "NY", "locality": borough.title()}, "BOROUGH_ONLY"))

    for query, comps, tag in cands:
        if not query:
            continue
        try:
            resp = gmaps.geocode(
                query,
                region="us",
                components=comps,
                bounds=NYC_BOUNDS
            )
        except Exception as e:
            print("Geocode error:", e)
            resp = []

        if resp:
            # prefer rooftop or range_interpolated results
            best = sorted(resp, key=lambda r: {"ROOFTOP":0, "RANGE_INTERPOLATED":1}.get(r.get("geometry",{}).get("location_type",""), 2))[0]
            loc = best["geometry"]["location"]
            return loc["lat"], loc["lng"], tag, best.get("formatted_address"), best.get("place_id")

    return None, None, "NO_HIT", None, None

# assumes without_coord is a DataFrame with the relevant columns
for i, r in without_coord[["property_id","geocode_key","address_line1_clean","address_line2_clean","borough_clean","city_clean","county"]].drop_duplicates().iterrows():
    # keep your i >= guard if needed
    temp_df = pd.DataFrame([r])
    lat, lng, tag, faddr, pid = geocode_row(r)

    if lat is not None and lng is not None:
        temp_df["latitude"]  = lat
        temp_df["longitude"] = lng
        temp_df["geocode_tag"] = tag
        temp_df["formatted_address"] = faddr
        temp_df["place_id"] = pid
        results.append(temp_df)
    else:
        temp_df["geocode_tag"] = tag  # e.g., POBOX, MISSING_STREET, NO_HIT
        failed.append(temp_df)


In [208]:


# # assumes: without_coord, results = [], failed = [] already exist
# for i, r in without_coord[["property_id","geocode_key"]].drop_duplicates().iterrows():
#     if i >= 121392:
#         temp_df = pd.DataFrame([r])
#         print(r["property_id"])

#         full_address = r["geocode_key"]
#         print(full_address)

#         if "/" in full_address:
#             full_address = full_address.split("/")[-1].strip()
#             print("Split Addy:", full_address)

#         # ---- Google Geocoding (swap-in for geolocator.geocode) ----
#         try:
#             resp = gmaps.geocode(full_address)  # add region="us" or components=... if you like
#         except Exception as e:
#             print("Geocode error:", e)
#             resp = []

#         if resp:
#             loc = resp[0]["geometry"]["location"]
#             temp_df["latitude"]  = loc["lat"]
#             temp_df["longitude"] = loc["lng"]
#             print(loc["lat"], loc["lng"])
#             results.append(temp_df)
#         else:
#             failed.append(temp_df)


3089718.0
84-17 125TH ST. ET. AL.||QUEENS|NY|
40.7072689 -73.826502
3108051.0
3400 TYRON AVE||BRONX|NY|
40.87956519999999 -73.8767307
3116937.0
854 WEST 181ST||NEW YORK|NY|
40.8511379 -73.9399545
3128084.0
||BROOKLYN|NY|
40.6781784 -73.9441579
3128087.0
||BROOKLYN|NY|
40.6781784 -73.9441579
3128113.0
||BRONX|NY|
40.8447819 -73.8648268
3128115.0
NA NA||BRONX|NY|
40.8447819 -73.8648268
3128128.0
||BRONX|NY|
40.8447819 -73.8648268
3128739.0
142-18 38TH ST||QUEENS|NY|
40.7616081 -73.82639309999999
3129467.0
301 EAST 22ND||NEW YORK|NY|
40.7371253 -73.9806613
3129938.0
54 WEST 94TH ST||NEW YORK|NY|
40.7908594 -73.9673689
3129939.0
||BROOKLYN|NY|
40.6781784 -73.9441579
3223085.0
152-154-156 EAST 171 ST||BRONX|NY|
40.8400493 -73.914232
3234049.0
3052-3054 KINGS BRIDGE AVE||BRONX|NY|
40.8794388 -73.9065129
3464982.0
1464-66, 72, 74 WATSON AVE||BRONX|NY|
40.8280409 -73.8636811
3465047.0
1484-86-92-94 WATSON AVE||BRONX|NY|
40.8255082 -73.8798845
3503004.0
18 34TH RD||QUEENS|NY|
40.7691161 -73.775

In [229]:
print(len(results))
print(len(failed)) ## Zero failures

583
0


In [230]:
# reading in past resu;ts
results_df = pd.concat(results)

In [233]:
without_coord_geo = without_coord.dropna(how='all',axis=1).merge(results_df, 
                                                                 on=["property_id","geocode_key","address_line1_clean","address_line2_clean","borough_clean","city_clean","county"],
                                                                 how='left')
without_coord_geo#[without_coord_geo['latitude'].isnull()]

,property_id,address_1,address_2,city,county,borough,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,needs_geocoding,geocode_key,latitude,longitude,geocode_tag,formatted_address,place_id
0,2638790.0,43-22,NaN,Sunnyside,NaN,NaN,43-22,None,QUEENS,Sunnyside,NY,True,43-22||SUNNYSIDE|NY|,40.728224,-73.794852,BOROUGH_ONLY,"Queens, NY, USA",ChIJK1kKR2lDwokRBXtcbIvRCUE
1,3524527.0,150-74th Street,NaN,Brooklyn,NaN,NaN,150-74Th St,None,BROOKLYN,Brooklyn,NY,True,150-74TH ST||BROOKLYN|NY|,40.678178,-73.944158,BOROUGH_ONLY,"Brooklyn, NY, USA",ChIJCSF8lBZEwokRhngABHRcdoI
2,2638326.0,308 West104,NaN,New York,NaN,NaN,308 West 104Th St,None,MANHATTAN,New York,NY,True,308 WEST 104TH ST||NEW YORK|NY|,40.768517,-73.982194,BOROUGH_ONLY,"Manhattan, New York, NY, USA",ChIJYeZuBI9YwokRjMDs_IEyCwo
3,2782781.0,106 & 114 West 143rd Street,NaN,New York,NaN,NaN,106 & 114 West 143Rd St,None,MANHATTAN,New York,NY,True,106 & 114 WEST 143RD ST||NEW YORK|NY|,40.768517,-73.982194,BOROUGH_ONLY,"Manhattan, New York, NY, USA",ChIJYeZuBI9YwokRjMDs_IEyCwo
4,2946930.0,777/783 Fox Street,NaN,Bronx,NaN,NaN,783 Fox St,None,BRONX,Bronx,NY,True,783 FOX ST||BRONX|NY|,40.844782,-73.864827,BOROUGH_ONLY,"Bronx, NY, USA",ChIJsXxpOlWLwokRd1zxj6dDblU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,4125392.0,720 West 11th Street,NaN,New York,NaN,NaN,720 West 11Th St,None,MANHATTAN,New York,NY,True,720 WEST 11TH ST||NEW YORK|NY|,40.768517,-73.982194,BOROUGH_ONLY,"Manhattan, New York, NY, USA",ChIJYeZuBI9YwokRjMDs_IEyCwo
582,2642595.0,3099 brighton 6thStreet,NaN,brooklyn,kings,NaN,3099 Brighton 6Thstreet,None,BROOKLYN,Brooklyn,NY,True,3099 BRIGHTON 6THSTREET||BROOKLYN|NY|,40.678178,-73.944158,BOROUGH_ONLY,"Brooklyn, NY, USA",ChIJCSF8lBZEwokRhngABHRcdoI
583,2647976.0,9002-9046 153rd. ave 9001-9055 Shore Parkway,Lindenwood,New York,NaN,NaN,9002-9046 153Rd. Ave 9001-9055 Shore Pkwy,APT LINDENWOOD,MANHATTAN,New York,NY,True,9002-9046 153RD. AVE 9001-9055 SHORE PKWY|APT ...,40.768517,-73.982194,BOROUGH_ONLY,"Manhattan, New York, NY, USA",ChIJYeZuBI9YwokRjMDs_IEyCwo
584,3579396.0,1990 Adam Clayton Powell Jr Boulevard,NaN,New York,NaN,NaN,1990 Adam Clayton Powell Jr Blvd,None,MANHATTAN,New York,NY,True,1990 ADAM CLAYTON POWELL JR BLVD||NEW YORK|NY|,40.768517,-73.982194,BOROUGH_ONLY,"Manhattan, New York, NY, USA",ChIJYeZuBI9YwokRjMDs_IEyCwo


In [234]:
 building_info_geocoded= pd.concat([with_coord,without_coord_geo])
# building_info_geocoded.to_csv("building_info_geocoded.csv",index=False)

In [4]:
building_info_geocoded = pd.read_csv("building_info_geocoded.csv")

In [5]:
### Limit the resutlsd based on Building-specific Lat Long, not borough centroids.
building_info_geocoded = building_info_geocoded[building_info_geocoded['geocode_tag'].isnull()]
### NEEED TO ADD THE BBL BY ADDRESS.
print(building_info_geocoded.shape)
# building_info_geocoded.to_csv("building_info_geocoded.csv",index=False)

(9121, 20)


In [3]:
# building_info_geocoded_bbl.to_csv("building_info_geocoded_bbl.csv",index=False)
# building_info_geocoded_bbl = pd.read_csv("building_info_geocoded_bbl.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'building_info_geocoded_bbl.csv'

In [30]:
import requests, time, re, pandas as pd

df = building_info_geocoded.copy()
rows = []

with requests.Session() as s:
    for i, r in df.iterrows():
        addr = str(r.get("address_line1_clean", "")).strip()
        boro = str(r.get("borough_clean", "")).strip()
        if not addr:
            rows.append((i, None, None, None, None, None)); continue

        q = f"{addr}, {boro}, NY" if boro else f"{addr}, NY"
        try:
            resp = s.get("https://geosearch.planninglabs.nyc/v2/search",
                         params={"text": q, "size": 5}, timeout=20)
            feats = resp.json().get("features", [])
            # pick the best feature: prefer layer=='address' and same borough; else highest confidence
            best, best_score = None, (-1, -1.0)
            for f in feats:
                p = f.get("properties", {})
                layer = p.get("layer")
                conf  = float(p.get("confidence", 0) or 0)
                fb    = (p.get("borough") or ((p.get("addendum") or {}).get("pad") or {}).get("boroughName") or "")
                score = (2 if layer == "address" else 0) + (1 if boro and boro.upper() in str(fb).upper() else 0)
                if (score, conf) > best_score:
                    best, best_score = f, (score, conf)

            if best:
                props = best.get("properties", {})
                add   = (props.get("addendum") or {}).get("pad") or {}
                bbl   = add.get("bbl") or props.get("bbl")
                bin_  = add.get("bin") or props.get("bin")

                # normalize BBL to a 10-digit string
                if isinstance(bbl, str):
                    digits = re.sub(r"\D", "", bbl)
                    bbl = digits.zfill(10) if digits else None
                elif bbl is not None:
                    try: bbl = str(int(bbl)).zfill(10)
                    except: bbl = None

                rows.append((i, bbl, bin_, props.get("confidence"), props.get("label") or props.get("name"), q))
            else:
                rows.append((i, None, None, None, None, q))

        except Exception as e:
            rows.append((i, None, None, None, f"ERR:{type(e).__name__}", q))

        time.sleep(0.1)  # tiny throttle to be polite

# assemble + join back
out = pd.DataFrame(rows, columns=["__idx","BBL_from_api","BIN_from_api","confidence","match_label","query"]).set_index("__idx")


In [31]:
out

,BBL_from_api,BIN_from_api,confidence,match_label,query
__idx,,,,,
0,3074560006,3204886,0.8,"2626 HOMECREST AVENUE, Brooklyn, NY, USA","2626 Homecrest Ave, BROOKLYN, NY"
1,2057890024,2084124,0.8,"3240 HENRY HUDSON PARKWAY, Bronx, NY, USA","3240 Henry Hudson Pkwy, BRONX, NY"
2,3065120001,3170547,0.8,"900 AVENUE H, Brooklyn, NY, USA","900 Ave H, BROOKLYN, NY"
3,3067600062,3181781,0.8,"1561 EAST 13 STREET, Brooklyn, NY, USA","1561 E. 13Th St, BROOKLYN, NY"
4,1008117502,1015226,0.8,"115 WEST 35 STREET, New York, NY, USA","115 West 35Th St, MANHATTAN, NY"
...,...,...,...,...,...
9116,2027137501,2092007,0.8,"955 EAST 163 STREET, Bronx, NY, USA","955 East 163Rd St, MANHATTAN, NY"
9117,2028730103,2008741,0.8,"1541 SHAKESPEARE AVENUE, Bronx, NY, USA","1541 Shakespeare Ave, MANHATTAN, NY"
9118,1019120043,1089103,0.8,"128 WEST 128 STREET, New York, NY, USA","128 West 128Th St, MANHATTAN, NY"


In [7]:
# BuildingInfoGeocoded = building_info_geocoded.join(out)
# BuildingInfoGeocoded.to_csv("BuildingInfoGeocoded_GEO_BBL.csv",index=False)
BuildingInfoGeocoded =pd.read_csv("BuildingInfoGeocoded_GEO_BBL.csv")

# quick peek
BuildingInfoGeocoded[["address_line1_clean","borough_clean","BBL_from_api","BIN_from_api","confidence","match_label"]].head()


,address_line1_clean,borough_clean,BBL_from_api,BIN_from_api,confidence,match_label
0,2626 Homecrest Ave,BROOKLYN,3.074560e+09,3204886.0,0.8,"2626 HOMECREST AVENUE, Brooklyn, NY, USA"
1,3240 Henry Hudson Pkwy,BRONX,2.057890e+09,2084124.0,0.8,"3240 HENRY HUDSON PARKWAY, Bronx, NY, USA"
2,900 Ave H,BROOKLYN,3.065120e+09,3170547.0,0.8,"900 AVENUE H, Brooklyn, NY, USA"
3,1561 E. 13Th St,BROOKLYN,3.067600e+09,3181781.0,0.8,"1561 EAST 13 STREET, Brooklyn, NY, USA"
4,115 West 35Th St,MANHATTAN,1.008118e+09,1015226.0,0.8,"115 WEST 35 STREET, New York, NY, USA"


In [8]:
BuildingInfoGeocoded = BuildingInfoGeocoded.dropna(how='all',axis=1)

In [38]:
BuildingInfoGeocoded#.columns

,property_id,address_1,address_2,city,county,borough,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,needs_geocoding,geocode_key,BBL_from_api,BIN_from_api,confidence,match_label,query
0,2707907.0,2626 Homecrest Avenue,NaN,Brooklyn,NaN,BROOKLYN,40.587065,-73.957019,2626 Homecrest Ave,NaN,BROOKLYN,Brooklyn,NY,False,2626 HOMECREST AVE||BROOKLYN|NY|,3074560006,3204886,0.8,"2626 HOMECREST AVENUE, Brooklyn, NY, USA","2626 Homecrest Ave, BROOKLYN, NY"
1,3521602.0,3240 Henry Hudson parkway,NaN,Bronx,NaN,BRONX,40.885365,-73.913345,3240 Henry Hudson Pkwy,NaN,BRONX,Bronx,NY,False,3240 HENRY HUDSON PKWY||BRONX|NY|,2057890024,2084124,0.8,"3240 HENRY HUDSON PARKWAY, Bronx, NY, USA","3240 Henry Hudson Pkwy, BRONX, NY"
2,3521883.0,900 Avenue H,NaN,Brooklyn,NaN,BROOKLYN,40.629342,-73.967861,900 Ave H,NaN,BROOKLYN,Brooklyn,NY,False,900 AVE H||BROOKLYN|NY|,3065120001,3170547,0.8,"900 AVENUE H, Brooklyn, NY, USA","900 Ave H, BROOKLYN, NY"
3,3522892.0,1561 E. 13th Street,NaN,Brooklyn,NaN,BROOKLYN,40.611279,-73.960634,1561 E. 13Th St,NaN,BROOKLYN,Brooklyn,NY,False,1561 E. 13TH ST||BROOKLYN|NY|,3067600062,3181781,0.8,"1561 EAST 13 STREET, Brooklyn, NY, USA","1561 E. 13Th St, BROOKLYN, NY"
4,4047231.0,115 West 35th Street,NaN,new York,NaN,MANHATTAN,40.750878,-73.988270,115 West 35Th St,NaN,MANHATTAN,New York,NY,False,115 WEST 35TH ST||NEW YORK|NY|,1008117502,1015226,0.8,"115 WEST 35 STREET, New York, NY, USA","115 West 35Th St, MANHATTAN, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9116,3632773.0,955 East 163rd Street,NaN,New York,NaN,BRONX,40.821001,-73.895580,955 East 163Rd St,NaN,MANHATTAN,New York,NY,False,955 EAST 163RD ST||NEW YORK|NY|,2027137501,2092007,0.8,"955 EAST 163 STREET, Bronx, NY, USA","955 East 163Rd St, MANHATTAN, NY"
9117,3632791.0,1541 Shakespeare Avenue,NaN,New York,NaN,BRONX,40.845719,-73.919252,1541 Shakespeare Ave,NaN,MANHATTAN,New York,NY,False,1541 SHAKESPEARE AVE||NEW YORK|NY|,2028730103,2008741,0.8,"1541 SHAKESPEARE AVENUE, Bronx, NY, USA","1541 Shakespeare Ave, MANHATTAN, NY"
9118,3633523.0,128 West 128th Street,NaN,New York,NaN,MANHATTAN,40.810108,-73.945088,128 West 128Th St,NaN,MANHATTAN,New York,NY,False,128 WEST 128TH ST||NEW YORK|NY|,1019120043,1089103,0.8,"128 WEST 128 STREET, New York, NY, USA","128 West 128Th St, MANHATTAN, NY"
9119,3633666.0,315 E 102 Street,NaN,New York,NaN,MANHATTAN,40.787834,-73.943766,315 E 102St St,NaN,MANHATTAN,New York,NY,False,315 E 102ST ST||NEW YORK|NY|,1003500061,1004271,1.0,"315 EAST HOUSTON STREET, New York, NY, USA","315 E 102St St, MANHATTAN, NY"


# ---------------------------------

In [ ]:
BuildingInfoGeocoded

In [ ]:
START HERE:
- 

### Pluto Data for Num Floors and Height etc`

In [ ]:
### REading in the pluta data i need. Total thing crashed the kernel, limiting by lat long in the 

In [16]:
import time
import math
import re
import pandas as pd
import pyogrio
# SOURCE # https://www.nyc.gov/content/planning/pages/resources/datasets/mappluto-pluto-change#mappluto
# ===================== CONFIG =====================
SRC   = r"C:/Users/johnf/Downloads/nyc_mappluto_25v3_fgdb/MapPLUTO25v3.gdb"
LAYER = "MapPLUTO_25v3_clipped"

# PLUTO columns you need (no geometry necessary)
PLUTO_COLS = ["BBL","LandUse","NumFloors","UnitsRes","BldgClass","Block","Borough","BoroCode"]

# How many distinct BLOCKS per chunk (per borough)
BLOCKS_PER_CHUNK = 2000  # tune: 1000–4000 are typical sweet spots

# ===================== HELPERS =====================
def norm_bbl_series(s: pd.Series) -> pd.Series:
    """Normalize to 10-digit BBL strings; non-digits removed; blanks -> NA."""
    s = s.astype(str).str.replace(r"\D", "", regex=True).str.zfill(10)
    return s.mask(s.eq("0000000000"))

def boro_code_from_bbl_str(bbl: str) -> int:
    """First digit of BBL is borough code 1..5."""
    return int(bbl[0]) if isinstance(bbl, str) and bbl and bbl[0].isdigit() else None

def block_from_bbl_str(bbl: str) -> int:
    """Digits 2..6 (5 digits) are block."""
    try:
        return int(bbl[1:6])
    except Exception:
        return None

def pull_where(where: str, cols: list[str]) -> pd.DataFrame:
    """Run filtered attribute read; returns pandas.DataFrame (no geometry)."""
    return pyogrio.read_dataframe(
        SRC, layer=LAYER,
        columns=[c for c in cols if c != "geometry"],
        read_geometry=False,
        where=where,
    )

# ===================== 1) LL84 BBL PREP =====================
ll84 = BuildingInfoGeocoded.copy()
ll84["BBL_from_api"] = norm_bbl_series(ll84["BBL_from_api"])
ll84 = ll84[ll84["BBL_from_api"].notna()].copy()

# Build fast-lookup set and also borough/block lists (to define PLUTO chunks)
bbl_set = set(ll84["BBL_from_api"].unique())
print(f"[prep] Unique LL84 BBLs: {len(bbl_set):,}")

ll84["boro_code"] = ll84["BBL_from_api"].map(boro_code_from_bbl_str)
ll84["block_num"] = ll84["BBL_from_api"].map(block_from_bbl_str)

# Map of borough code -> sorted unique blocks from LL84 (what we care about)
blocks_by_boro = (
    ll84.dropna(subset=["boro_code","block_num"])
        .groupby("boro_code")["block_num"]
        .apply(lambda s: sorted(set(s.astype(int))))
        .to_dict()
)

# ===================== 2) DETECT PLUTO FIELD NAMES =====================
# We’ll try to discover which borough/block fields exist by fetching a tiny sample via WHERE on a few LL84 BBLs.
sample_bbls = list(bbl_set)[:25]
in_list = ",".join(f"'{bb}'" for bb in sample_bbls)
probe = pull_where(f"BBL IN ({in_list})", PLUTO_COLS)
pluto_cols = set(probe.columns)

# Choose Borough & Block columns if present
boro_field = None
for cand in ["BoroCode","BOROCODE","Borough","BOROUGH","boro","borough"]:
    if cand in pluto_cols:
        boro_field = cand
        break

block_field = None
for cand in ["Block","BLOCK","block"]:
    if cand in pluto_cols:
        block_field = cand
        break

have_boro_block = (boro_field is not None) and (block_field is not None)
print(f"[detect] PLUTO borough field: {boro_field} | block field: {block_field} | have_boro_block={have_boro_block}")

# ===================== 3) CHUNK PLUTO BY BORO + BLOCKS (preferred path) =====================
parts = []
t0 = time.time()

if have_boro_block and blocks_by_boro:
    # Map our boro_code 1..5 to the values PLUTO expects
    # If PLUTO has BoroCode numeric, we use 1..5; if 'Borough' text, map to strings.
    boro_map_text = {1:"MANHATTAN", 2:"BRONX", 3:"BROOKLYN", 4:"QUEENS", 5:"STATEN ISLAND"}

    # Build a list of all planned chunks so we can print chunk N / total
    plan = []
    for boro_code, blocks in blocks_by_boro.items():
        if not blocks: 
            continue
        for ci in range(0, len(blocks), BLOCKS_PER_CHUNK):
            plan.append( (boro_code, blocks[ci:ci+BLOCKS_PER_CHUNK]) )
    total_chunks = len(plan)
    print(f"[plan] Will pull {total_chunks} PLUTO chunks (by borough + block groups)")

    for idx, (boro_code, blocks_chunk) in enumerate(plan, start=1):
        # Borough clause
        if boro_field.lower() in ("borocode","borocode".upper()):
            # numeric match
            borocla = f"{boro_field} = {int(boro_code)}"
        else:
            # text match
            boroname = boro_map_text.get(int(boro_code), "")
            borocla = f"{boro_field} = '{boroname}'"

        # Block clause (IN list of ints)
        block_list = ",".join(str(int(b)) for b in blocks_chunk)
        where = f"{borocla} AND {block_field} IN ({block_list})"

        # Pull this PLUTO chunk (attributes only), then keep only BBLs in our LL84 set
        chunk_df = pull_where(where, PLUTO_COLS)
        if not chunk_df.empty:
            chunk_df["BBL"] = norm_bbl_series(chunk_df["BBL"])
            keep = chunk_df[chunk_df["BBL"].isin(bbl_set)]
            if not keep.empty:
                parts.append(keep)

        print(f"[pull] chunk {idx}/{total_chunks} | boro={boro_code} | blocks {blocks_chunk[0]}–{blocks_chunk[-1]} "
              f"| pulled {len(chunk_df):,} | kept {0 if chunk_df.empty else len(keep):,}")

else:
    # ===================== 3b) FALLBACK: chunk by BBL IN(...) batches =====================
    # If we can't address by borough+block, we batch BBLs directly (still chunked, zero full scans).
    bbls = list(bbl_set)
    BATCH = 800
    total_chunks = math.ceil(len(bbls) / BATCH)
    print(f"[plan] Fallback: {total_chunks} chunks via BBL IN(...) batches")

    # Detect whether BBL is TEXT or NUMERIC in this GDB (quoted vs unquoted)
    quote_as_text = True
    try:
        test = pull_where(f"BBL IN ('{bbls[0]}')", PLUTO_COLS)
        # if no exception, assume quoted works; we don't require >0 rows
        quote_as_text = True
    except Exception:
        quote_as_text = False

    for ci in range(total_chunks):
        chunk = bbls[ci*BATCH:(ci+1)*BATCH]
        in_list = ",".join(f"'{bb}'" for bb in chunk) if quote_as_text else ",".join(str(int(bb)) for bb in chunk)
        where = f"BBL IN ({in_list})"
        part = pull_where(where, PLUTO_COLS)
        if not part.empty:
            part["BBL"] = norm_bbl_series(part["BBL"])
            parts.append(part)
        print(f"[pull] chunk {ci+1}/{total_chunks} | BBLs {ci*BATCH+1:,}-{ci*BATCH+len(chunk):,} "
              f"of {len(bbls):,} | pulled {len(part):,}")

# ===================== 4) CONCAT MATCHES + MERGE BACK =====================
elapsed = time.time() - t0
pluto_by_bbl = (pd.concat(parts, ignore_index=True) if parts
                else pd.DataFrame(columns=PLUTO_COLS))
print(f"[done] Kept PLUTO rows: {len(pluto_by_bbl):,} in {elapsed:,.1f}s")

# Merge back to LL84 by BBL
ll84_enriched = ll84.merge(pluto_by_bbl.drop_duplicates("BBL"), left_on="BBL_from_api", right_on="BBL", how="left")

# Optional: derive stories & residential <5
ll84_enriched["stories"] = ll84_enriched["NumFloors"]
res_codes = {"01","02","03","04"}
is_res = (
    ll84_enriched["LandUse"].astype(str).isin(res_codes)
    | (ll84_enriched["UnitsRes"].fillna(0) > 0)
    | ll84_enriched.get("BldgClass", pd.Series(index=ll84_enriched.index, dtype="object"))
      .fillna("").str.startswith(("A","B"))
)
under5 = ll84_enriched[is_res & ll84_enriched["stories"].notna() & (ll84_enriched["stories"] < 5)].copy()
print(f"[summary] LL84 rows: {len(ll84_enriched):,} | Residential <5 stories: {len(under5):,}")


[prep] Unique LL84 BBLs: 8,165
[detect] PLUTO borough field: BoroCode | block field: Block | have_boro_block=True
[plan] Will pull 5 PLUTO chunks (by borough + block groups)
[pull] chunk 1/5 | boro=1 | blocks 7–2250 | pulled 29,744 | kept 3,286
[pull] chunk 2/5 | boro=2 | blocks 2267–5953 | pulled 19,613 | kept 1,553
[pull] chunk 3/5 | boro=3 | blocks 27–8821 | pulled 37,199 | kept 1,781
[pull] chunk 4/5 | boro=4 | blocks 15–16234 | pulled 20,642 | kept 1,456
[pull] chunk 5/5 | boro=5 | blocks 1–6694 | pulled 2,182 | kept 74
[done] Kept PLUTO rows: 8,150 in 221.2s
[summary] LL84 rows: 9,049 | Residential <5 stories: 721


In [19]:
ll84_enriched

,property_id,address_1,address_2,city,county,borough,latitude,longitude,address_line1_clean,address_line2_clean,...,block_num,BBL,BldgClass,Block,BoroCode,Borough,LandUse,NumFloors,UnitsRes,stories
0,2707907.0,2626 Homecrest Avenue,NaN,Brooklyn,NaN,BROOKLYN,40.587065,-73.957019,2626 Homecrest Ave,NaN,...,7456,30745600060,D4,7456.0,3.0,BK,03,6.0,140.0,6.0
1,3521602.0,3240 Henry Hudson parkway,NaN,Bronx,NaN,BRONX,40.885365,-73.913345,3240 Henry Hudson Pkwy,NaN,...,5789,20578900240,D1,5789.0,2.0,BX,03,6.0,108.0,6.0
2,3521883.0,900 Avenue H,NaN,Brooklyn,NaN,BROOKLYN,40.629342,-73.967861,900 Ave H,NaN,...,6512,30651200010,D1,6512.0,3.0,BK,03,6.0,65.0,6.0
3,3522892.0,1561 E. 13th Street,NaN,Brooklyn,NaN,BROOKLYN,40.611279,-73.960634,1561 E. 13Th St,NaN,...,6760,30676000620,D1,6760.0,3.0,BK,03,6.0,66.0,6.0
4,4047231.0,115 West 35th Street,NaN,new York,NaN,MANHATTAN,40.750878,-73.988270,115 West 35Th St,NaN,...,811,10081175020,RC,811.0,1.0,MN,05,12.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9044,3632773.0,955 East 163rd Street,NaN,New York,NaN,BRONX,40.821001,-73.895580,955 East 163Rd St,NaN,...,2713,20271375010,RR,2713.0,2.0,BX,04,8.0,268.0,8.0
9045,3632791.0,1541 Shakespeare Avenue,NaN,New York,NaN,BRONX,40.845719,-73.919252,1541 Shakespeare Ave,NaN,...,2873,20287301030,D7,2873.0,2.0,BX,04,6.0,72.0,6.0
9046,3633523.0,128 West 128th Street,NaN,New York,NaN,MANHATTAN,40.810108,-73.945088,128 West 128Th St,NaN,...,1912,10191200430,D1,1912.0,1.0,MN,03,6.0,80.0,6.0
9047,3633666.0,315 E 102 Street,NaN,New York,NaN,MANHATTAN,40.787834,-73.943766,315 E 102St St,NaN,...,350,10035000610,C3,350.0,1.0,MN,02,4.0,4.0,4.0


In [18]:
# ll84_enriched.to_csv("building_go_bbl_pluto_enr.csv",index=False)

### Spatial joinign with CT to pull ct into the buildings data. 

In [ ]:
### pulling in ct shapefile

In [46]:
import io, requests, geopandas as gpd

CT2010_URL = "https://data.cityofnewyork.us/resource/bmjq-373p.geojson?$limit=50000"
SOCRATA_APP_TOKEN = None  # put your token string here if you have one

headers = {"X-App-Token": SOCRATA_APP_TOKEN} if SOCRATA_APP_TOKEN else {}
resp = requests.get(CT2010_URL, headers=headers, timeout=60)
resp.raise_for_status()

ct2010 = gpd.read_file(io.BytesIO(resp.content)).to_crs(2263)
ct2010.head()


,ntacode,shape_area,ntaname,shape_leng,boroname,puma,boroct2010,ct2010,borocode,cdeligibil,ctlabel,geometry
0,SI22,2497009.71359,West New Brighton-New Brighton-St. George,7729.01679383,Staten Island,3903,5000900,000900,5,E,9,"MULTIPOLYGON (((962269.126 173705.5, 962288.72..."
1,MN17,1860992.68163,Midtown-Midtown South,5687.80243891,Manhattan,3807,1010200,010200,1,I,102,"MULTIPOLYGON (((992216.539 216507.687, 992091...."
2,MN17,1864600.43538,Midtown-Midtown South,5693.03636707,Manhattan,3807,1010400,010400,1,I,104,"MULTIPOLYGON (((991325.882 217001.689, 991199...."
3,MN17,1890907.25105,Midtown-Midtown South,5699.86064037,Manhattan,3807,1011300,011300,1,I,113,"MULTIPOLYGON (((988650.277 214286.402, 988517...."
4,MN40,1918144.56374,Upper East Side-Carnegie Hill,5807.97295649,Manhattan,3805,1013000,013000,1,I,130,"MULTIPOLYGON (((994920.11 221386.27, 994791.85..."


In [47]:
import geopandas as gpd

# --- 1) Points from your geocoded table ---
# assumes working_residential_geo has columns: property_id, latitude, longitude
pts = gpd.GeoDataFrame(
    ll84_enriched[["property_id", "latitude", "longitude"]].copy(),
    geometry=gpd.points_from_xy(
        ll84_enriched["longitude"], ll84_enriched["latitude"]
    ),
    crs=4326  # your geocodes are WGS84
)

# project points to match the CT layer (your CT layer is already to_crs(2263))
pts_2263 = pts.to_crs(2263)

# --- 2) Keep only what you need from the CT layer ---
# 'ct2010' and/or 'boroct2010' are the usual tract IDs in that NYC layer
ct_keep = ct2010[["ct2010", "boroct2010", "boroname", "geometry"]].copy()

# --- 3) Spatial join: which tract polygon contains each point ---
joined = gpd.sjoin(
    pts_2263,
    ct_keep,
    how="left",
    predicate="within"   # points that fall inside a tract polygon
)

# If a property_id appears multiple times (duplicates), keep the first tract hit
joined = joined.sort_index().drop_duplicates(subset=["property_id"])

# --- 4) Bring tract columns back to your original dataframe ---
cols_to_add = ["ct2010", "boroct2010", "boroname"]
ll84_enriched_ct = ll84_enriched.merge(
    joined[["property_id"] + cols_to_add],
    on="property_id",
    how="left"
)

# working_with_tract now has ct2010 / boroct2010 for each row


In [49]:
# ll84_enriched_ct.to_csv("ll84_enriched_ct.csv",index=False)

In [18]:
## Saving information to Csv 
# working_residential_tract.to_csv("working_residential_tract.csv",index=False)
# working_residential_tract = pd.read_csv("working_residential_tract.csv")

C:\Users\johnf\AppData\Local\Temp\ipykernel_42220\1074909641.py:3: DtypeWarning: Columns (4,5,8,14,18,19,20,43,44,45,46,47,53,54,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,98,112,118,126,127,128,129,130,131,132,133,138,139,140,146,147,148,151,154,155,156,160,163,172,173,175,176,177,178,179,180,181,183,185,188,189,190,210,211,231,245,247,248,249,250,251,252,253,254,255,256,257,258,259,260,272,273,274,275,276,277,281,282,286,289,295,296,303,304,305,306,307,308,309,318,319,333,334,335,336,337,339,342,345,347,348,352) have mixed types. Specify dtype option on import or set low_memory=False.
  working_residential_tract = pd.read_csv("working_residential_tract.csv")


In [ ]:
## limiting to certain 2010 and 2017?
ll84_enriched_ct

In [85]:
# Clean Borrough does not equal boro
fine = ll84_enriched_ct[((ll84_enriched_ct["borough"]==ll84_enriched_ct["borough_clean"])|
                         (ll84_enriched_ct["borough_clean"]=='STATEN ISLAND'))]
need_examination  = ll84_enriched_ct[((ll84_enriched_ct["borough"]!=ll84_enriched_ct["borough_clean"])
                        # S.I. is fine, manually checked excluding that brorough from ehre
                         &(ll84_enriched_ct["borough_clean"]!='STATEN ISLAND'))]
print(need_examination.shape)
need_examination.groupby(["borough","borough_clean","boro_code"]).agg({"property_id":"count"})

(132, 34)


property_id
borough  borough_clean boro_code             
BRONX    MANHATTAN     1                    9
                       2                   60
BROOKLYN MANHATTAN     1                    8
                       3                   18
QUEENS   BROOKLYN      4                    2
         MANHATTAN     1                    8
                       4                   27

In [102]:
#Bronx Keeping
bronx_keeping = need_examination[((need_examination['borough']=="BRONX")
      &(need_examination["borough_clean"]=='MANHATTAN')
                 &(need_examination["match_label"].astype(str).str.contains("Bronx")))]
print(bronx_keeping.shape) ## Droping 4 instances of wrong entirs
#Brooklyn Keeping
brooklyn_keeping = need_examination[((need_examination['borough']=="BROOKLYN")
      &(need_examination["borough_clean"]=='MANHATTAN')
        &(need_examination["match_label"].astype(str).str.contains("Brooklyn")))]
print(brooklyn_keeping.shape)### Dropping 8 buildings
# QUEENS
queens_keeping1 = need_examination[((need_examination['borough']=="QUEENS")
      &(need_examination["borough_clean"]=='BROOKLYN')
      &(need_examination["match_label"].astype(str).str.contains("Far Rockaway")))]
queens_keeping2 = need_examination[((need_examination['borough']=="QUEENS")
      &(need_examination["borough_clean"]=='MANHATTAN')
                 &(need_examination["boro_code"]==4))]

## Dropping ~8 cols

(65, 34)
(18, 34)


In [129]:
examined_fine = pd.concat([bronx_keeping,brooklyn_keeping,queens_keeping1,queens_keeping2])
print(examined_fine.shape) ### overall dro[ping 20 rows
ll84_enriched_ct_clean1 = pd.concat([examined_fine,fine])

(112, 34)


In [130]:
pd.options.display.max_columns = 5400
## cleaning up the completely enriched building dataset
print(ll84_enriched_ct_clean1["confidence"].unique()) ### 80% and above so its all fine, and i can drop




### 
# ll84_enriched_ct

ll84_enriched_ct_clean1["raw_clean_address"] = ll84_enriched_ct_clean1["query"].combine_first(ll84_enriched_ct_clean1["geocode_key"])
ll84_enriched_ct_clean1["bbl"] = ll84_enriched_ct_clean1["BBL"].combine_first(ll84_enriched_ct_clean1["BBL_from_api"])
ll84_enriched_ct_clean1["address_line1_clean"] = ll84_enriched_ct_clean1["address_line1_clean"].combine_first(ll84_enriched_ct_clean1["address_1"])
ll84_enriched_ct_clean1["address_line2_clean"] = ll84_enriched_ct_clean1["address_line2_clean"].combine_first(ll84_enriched_ct_clean1["address_2"])
ll84_enriched_ct_clean1["city_clean"] = ll84_enriched_ct_clean1["city"].combine_first(ll84_enriched_ct_clean1["city_clean"])
ll84_enriched_ct_clean1["borough_clean"] = ll84_enriched_ct_clean1["borough"].combine_first(ll84_enriched_ct_clean1["borough_clean"])



to_drop = ["needs_geocoding","confidence","geocode_key","query","BBL_from_api",
           "BBL","BoroCode","Borough","boroname","block_num",
           "address_1", "address_2","city","borough","county"]
ll84_enriched_ct_clean2 = ll84_enriched_ct_clean1.drop(columns=to_drop)
ll84_enriched_ct_clean2 = ll84_enriched_ct_clean2.rename(columns = {"BIN_from_api":"BIN","match_label":"api_matched_address"})

[0.8 1. ]


In [132]:
# ll84_enriched_ct_clean2.to_csv("ll84_enriched_ct_clean2.csv",index=False)

In [134]:
ll84_enriched_ct_clean2.shape
### ~ 9,000 buildign universe to work with, pivoting to weaving in the canoopy tree data 

(9029, 21)

In [164]:
### last Trimming for BBL borocode not matching
##### WHILE Acriss seemed to confrim some of these were correct,
##### im being overtly cautious to have proper data. The borocode in the BBL shoudl match the location 
ll84_enriched_ct_clean3 = ll84_enriched_ct_clean2[(
    ## BRONX
    ((ll84_enriched_ct_clean2["borough_clean"]=="BRONX")
                         &(ll84_enriched_ct_clean2["boro_code"]==2))
    ## BROOKLYN
    |((ll84_enriched_ct_clean2["borough_clean"]=="BROOKLYN")
                         &(ll84_enriched_ct_clean2["boro_code"]==3))
    ## QUEENS
    |((ll84_enriched_ct_clean2["borough_clean"]=="QUEENS")
                         &(ll84_enriched_ct_clean2["boro_code"]==4))
        ## Statent Island
    |((ll84_enriched_ct_clean2["borough_clean"]=="STATEN IS")
                         &(ll84_enriched_ct_clean2["boro_code"]==5))
    ## Manhattan
    |((ll84_enriched_ct_clean2["borough_clean"]=="MANHATTAN")
                         &(ll84_enriched_ct_clean2["boro_code"]==1))
)]
ll84_enriched_ct_clean3["borough_clean"][ll84_enriched_ct_clean3["borough_clean"]=="STATEN IS"]="STATEN ISLAND"

C:\Users\johnf\AppData\Local\Temp\ipykernel_36636\3792486778.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ll84_enriched_ct_clean3["borough_clean"][ll84_enriched_ct_clean3["borough_clean"]=="STATEN IS"]="STATEN ISLAND"
C:\Users\johnf\

In [160]:
ll84_enriched_ct_clean2[((ll84_enriched_ct_clean2["borough_clean"]=="STATENISLAND"))]
                         # &(ll84_enriched_ct_clean2["boro_code"]==5))]

,property_id,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,BIN,api_matched_address,boro_code,BldgClass,Block,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,raw_clean_address,bbl


In [161]:
ll84_enriched_ct_clean2
ll84_enriched_ct_clean2["borough_clean"].unique()

array(['BRONX', 'BROOKLYN', 'QUEENS', 'MANHATTAN', 'STATEN IS'],
      dtype=object)

In [166]:
ll84_enriched_ct_clean3.to_csv("ll84_enriched_ct_clean3.csv",index=False)
ll84_enriched_ct_clean3.shape

(8986, 21)

In [165]:
### 8800 is my final working set with confidence. 
ll84_enriched_ct_clean3.groupby(["borough_clean","boro_code"]).agg({"property_id":"count"})

,,property_id
borough_clean,boro_code,
BRONX,2,1709
BROOKLYN,3,1964
MANHATTAN,1,3479
QUEENS,4,1735
STATEN ISLAND,5,99


### Tree Canopy Joining via CT

In [155]:
# Path to your geodatabase (.gdb)
gdb_path = r"C:\Users\johnf\Downloads\Tree_Canopy_Change (1)\Tree_Canopy_Change\NYC_TreeCanopyChange_2010_2017.gdb"

# List all layers in the geodatabase
layers = fiona.listlayers(gdb_path)
print("Layers available:", layers)

# Read a specific layer into a GeoDataFrame
gdf = gpd.read_file(gdb_path, layer=layers[0])  # Replace [0] with your desired layer
print(gdf.head())

# Optional: save as shapefile or GeoJSON
# gdf.to_file("output.shp") 
# gdf.to_file("output.geojson", driver="GeoJSON")

canopy_change = gdf.copy()
canopy_change

Layers available: ['NYC_TreeCanopyChange_2010_2017']
       Class  Shape_Length    Shape_Area  \
0  No Change    732.955182   5800.500001   
1  No Change   2079.456465  35005.875002   
2  No Change    228.557599   1984.500000   
3  No Change    125.726893    617.750000   
4  No Change     41.031601     90.750000   

                                            geometry  
0  MULTIPOLYGON (((1008879.93 272372.3, 1008878.4...  
1  MULTIPOLYGON (((1008749.43 271870.8, 1008748.9...  
2  MULTIPOLYGON (((1008704.43 272105.3, 1008701.9...  
3  MULTIPOLYGON (((1009015.43 272757.3, 1009013.4...  
4  MULTIPOLYGON (((1008794.93 270669.8, 1008781.4...  


,Class,Shape_Length,Shape_Area,geometry
0,No Change,732.955182,5800.500001,"MULTIPOLYGON (((1008879.93 272372.3, 1008878.4..."
1,No Change,2079.456465,35005.875002,"MULTIPOLYGON (((1008749.43 271870.8, 1008748.9..."
2,No Change,228.557599,1984.500000,"MULTIPOLYGON (((1008704.43 272105.3, 1008701.9..."
3,No Change,125.726893,617.750000,"MULTIPOLYGON (((1009015.43 272757.3, 1009013.4..."
4,No Change,41.031601,90.750000,"MULTIPOLYGON (((1008794.93 270669.8, 1008781.4..."
...,...,...,...,...
5692504,Loss,63.798210,286.735284,"MULTIPOLYGON (((1014610.46 188155.382, 1014610..."
5692505,Loss,63.429348,314.238467,"MULTIPOLYGON (((1014606.12 187969.835, 1014605..."
5692506,Loss,223.829388,2467.258600,"MULTIPOLYGON (((1014946.295 187646.6, 1014944...."
5692507,Loss,96.181802,538.668512,"MULTIPOLYGON (((1014656.958 187618.818, 101465..."


### Shaptial Join for LiDar Canopy and Buildings Lat Long for those in thwe Data from 2012/2013 through 2017

In [167]:
# --- 1) Points from your buildings (WGS84) ---
shape_bldg = gpd.GeoDataFrame(
    ll84_enriched_ct_clean3[["property_id", "latitude", "longitude"]].copy(),
    geometry=gpd.points_from_xy(
        ll84_enriched_ct_clean3["longitude"], ll84_enriched_ct_clean3["latitude"]
    ),
    crs=4326
)

# --- 2) Put both layers in the SAME CRS (use the canopy CRS) ---
canopy = canopy_change[["Class", "geometry"]].copy()
if canopy.crs is None:
    # set if your canopy file didn’t come with a CRS; adjust if needed
    canopy = canopy.set_crs(4326)  # NYC StatePlane ft; change if different

shape_bldg = shape_bldg.to_crs(canopy.crs)

# (optional but recommended) fix invalid polygon rings that can break joins
canopy["geometry"] = canopy.buffer(0)

In [168]:
# --- 3) Spatial join: which canopy polygon contains each building point? ---
joined = gpd.sjoin(
    shape_bldg[["property_id", "geometry"]],
    canopy,
    how="left",
    predicate="within"
)

In [169]:
joined

,property_id,geometry,index_right,Class
66,2740249.0,POINT (1024512.899 245703.232),NaN,NaN
67,2740271.0,POINT (1024517.876 245705.062),NaN,NaN
68,2741284.0,POINT (1023305.052 245062.207),NaN,NaN
69,2741286.0,POINT (1023771.065 245373.014),612188.0,Loss
70,2741298.0,POINT (1022925.093 244918.049),NaN,NaN
...,...,...,...,...
9038,3622045.0,POINT (1043223.071 226111.064),NaN,NaN
9039,3630489.0,POINT (990962.076 223589.066),NaN,NaN
9046,3633523.0,POINT (999450.896 234426.03),NaN,NaN
9047,3633666.0,POINT (999822.066 226311.068),986205.0,Gain


In [170]:
# If canopy polygons overlap and produce duplicates, keep the polygon with the largest area
if joined.duplicated("property_id").any():
    canopy_area = canopy.assign(_poly_area=canopy.area)
    joined = gpd.sjoin(
        shape_bldg[["property_id", "geometry"]],
        canopy_area,
        how="left",
        predicate="within"
    )
    joined = (joined
              .sort_values(["property_id", "_poly_area"], ascending=[True, False])
              .drop_duplicates(subset=["property_id"]))

In [171]:
# --- 4) Bring the canopy class back to your original table ---
ll84_enriched_ct_cln3_canopy = ll84_enriched_ct_clean3.merge(
    joined[["property_id", "Class"]].rename(columns={"Class": "canopy_change_class"}),
    on="property_id",
    how="left"
)


In [173]:
ll84_enriched_ct_cln3_canopy["canopy_change_class"].unique()

array([nan, 'Loss', 'Gain', 'No Change'], dtype=object)

In [174]:
ll84_enriched_ct_cln3_canopy.to_csv("ll84_enriched_ct_cln3_canopy.csv",index=False)

In [176]:
### Before snapping seeing how many data poitns are null , not have a canopy value
ll84_enriched_ct_cln3_canopy.groupby(["canopy_change_class"]).agg({"property_id":"nunique"})

,property_id
canopy_change_class,
Gain,674
Loss,134
No Change,953


In [177]:
# --- 5) fill misses by snapping to nearest polygon within 50 ft ---
# This helps when a point lands just outside a sliver polygon.
miss_ids = ll84_enriched_ct_cln3_canopy.loc[ll84_enriched_ct_cln3_canopy["canopy_change_class"].isna(), "property_id"]
if len(miss_ids):
    nearest = gpd.sjoin_nearest(
        shape_bldg[shape_bldg["property_id"].isin(miss_ids)],
        canopy,
        how="left",
        max_distance=50  # units = CRS units; 50ft if CRS=2263
    )[["property_id", "Class"]].rename(columns={"Class": "canopy_change_class_nearest"})

    ll84_enriched_ct_cln3_canopy_50ft = ll84_enriched_ct_cln3_canopy.merge(nearest, on="property_id", how="left")
    ll84_enriched_ct_cln3_canopy_50ft["canopy_change_class"] = (
        ll84_enriched_ct_cln3_canopy_50ft["canopy_change_class"]
        .fillna(ll84_enriched_ct_cln3_canopy_50ft["canopy_change_class_nearest"])
    )
    ll84_enriched_ct_cln3_canopy_50ft.drop(columns=["canopy_change_class_nearest"], inplace=True)

In [178]:
ll84_enriched_ct_cln3_canopy_50ft.groupby(["canopy_change_class"]).agg({"property_id":"nunique"})

,property_id
canopy_change_class,
Gain,6024
Loss,971
No Change,1368


In [179]:
ll84_enriched_ct_cln3_canopy_50ft.shape

(9563, 22)

In [180]:
ll84_enriched_ct_cln3_canopy_50ft = ll84_enriched_ct_cln3_canopy_50ft.dropna(how='all', axis=1)
ll84_enriched_ct_cln3_canopy_50ft.to_csv("working_with_canopy_50ft.csv",index=False)

In [182]:
ll84_enriched_ct_cln3_canopy_50ft

,property_id,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,BIN,api_matched_address,boro_code,BldgClass,Block,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,raw_clean_address,bbl,canopy_change_class
0,2740249.0,40.840982,-73.854486,1705 Purdy St,NaN,BRONX,New York,NY,2096760.0,"1705 PURDY STREET, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1705 Purdy St, MANHATTAN, NY",20394475010,Gain
1,2740271.0,40.840987,-73.854468,1718 Purdy St.,NaN,BRONX,New York,NY,2096761.0,"1718 PURDY STREET, Bronx, NY, USA",2,R4,3962.0,03,NaN,128.0,NaN,021001,2021001,"1718 Purdy St., MANHATTAN, NY",20396275010,Gain
2,2741284.0,40.839228,-73.858855,1501 Metropolitan Ave,NaN,BRONX,New York,NY,2096701.0,"1501 METROPOLITAN AVENUE, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1501 Metropolitan Ave, MANHATTAN, NY",20394475010,Gain
3,2741286.0,40.840079,-73.857169,1591 Metropolitan Ave,NaN,BRONX,New York,NY,2096768.0,"1591 METROPOLITAN AVENUE, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1591 Metropolitan Ave, MANHATTAN, NY",20394475010,Loss
4,2741298.0,40.838834,-73.860229,1 Metropolitan Oval,NaN,BRONX,New York,NY,2096667.0,"1 METROPOLITAN OVAL, Bronx, NY, USA",2,R4,3944.0,03,13.0,3857.0,13.0,021001,2021001,"1 Metropolitan Oval, MANHATTAN, NY",20394475010,Gain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9558,3622045.0,40.787102,-73.787038,13-34 Estates Ln,NaN,QUEENS,Queens,NY,4130109.0,"13-34 ESTATES LANE, North Bayside, NY, USA",4,P5,5843.0,08,2.0,0.0,2.0,099705,4099705,"13-34 Estates Ln, QUEENS, NY",40584300200,Loss
9559,3630489.0,40.780374,-73.975764,44 West 77Th St,NaN,MANHATTAN,New York,NY,1028821.0,"44 WEST 77 STREET, New York, NY, USA",1,D4,1129.0,03,14.5,32.0,14.5,016100,1016100,"44 West 77Th St, MANHATTAN, NY",10112900550,Gain
9560,3633523.0,40.810108,-73.945088,128 West 128Th St,NaN,MANHATTAN,New York,NY,1089103.0,"128 WEST 128 STREET, New York, NY, USA",1,D1,1912.0,03,6.0,80.0,6.0,022400,1022400,"128 West 128Th St, MANHATTAN, NY",10191200430,Gain
9561,3633666.0,40.787834,-73.943766,315 E 102St St,NaN,MANHATTAN,New York,NY,1004271.0,"315 EAST HOUSTON STREET, New York, NY, USA",1,C3,350.0,02,4.0,4.0,4.0,016400,1016400,"315 E 102St St, MANHATTAN, NY",10035000610,Gain


## Have BUilding Information and the Canopy Inforamation in ONe DF, need to join back wit hthe LL84 Energy year over Year data for final workign df

In [190]:
### limiting to the columns needed for analysis 
list(residential3_2012_2017.columns)
lim_residential3_2012_2017 = residential3_2012_2017[["property_id", "year_ending_year",
                                                     # Total energy (wearher-normalized) + aliases/fallbacks
                                                     "weather_normalized_site_energy_use_kbtu",  # preferred total
                                                     "weather_normalized_site_energy",           # legacy alias
                                                     "site_energy_use_kbtu",                     # non-normalized fallback
                                                     # Extra 
                                                     "property_gfa_calculated_buildings_ft",
                                                     "property_gfa_epa_calculated", #(if it is buildings-only in your vintage)
                                                     "property_gfa_self_reported_ft",
                                                     
                                                    "weather_normalized_site_eui_kbtu_ft",      # efficiency metric
                                                    "weather_normalized_site_eui",              # occasional alias
                                                    "number_of_active_energy_meters_used_to_compute_metrics",  # QA/backstop
                                                     "alert_energy_no_meters_selected_for_metrics",
                                                     "alert_energy_meter_has_less_than_12_full_calendar_months_of_data",
                                                     'alert_energy_meter_has_overlaps',
                                                     'alert_energy_meter_has_gaps',
                                                     'alert_energy_meter_has_single_entry_more_than_65_days',
                                                     'alert_energy_no_meters_selected_for_metrics'
                                                    ]]


In [209]:
building_energy_info_canopy.groupby(["year_ending_year"]).agg({"property_id":"count"})

,property_id
year_ending_year,
2012,39
2013,9669
2014,7956
2015,5039
2016,4452
2017,5637


In [207]:
### where is the main metric for energy use null
building_energy_info_canopy[((building_energy_info_canopy["weather_normalized_site_energy_use_kbtu"].isnull())
                             &(building_energy_info_canopy["weather_normalized_site_energy"].isnull())
                            )].shape

(6542, 38)

In [206]:
building_energy_info_canopy[((~building_energy_info_canopy["weather_normalized_site_energy_use_kbtu"].isnull())
                             |(~building_energy_info_canopy["weather_normalized_site_energy"].isnull())
                            )].shape

(26250, 38)

In [194]:

building_energy_info_canopy = lim_residential3_2012_2017.merge(ll84_enriched_ct_cln3_canopy_50ft, on = ["property_id"], how='inner')

In [197]:
## Working Data set for buioldings and energy. 
## Getting metrics
print(building_energy_info_canopy.shape)
## 
print("property _id")
print(len(building_energy_info_canopy["property_id"].unique()))
print("bbl")
print(len(building_energy_info_canopy["bbl"].unique()))

(32792, 38)
property _id
8635
bbl
8133


In [201]:
## Checking the property id / bbl relatinoship
test = building_energy_info_canopy.groupby(['bbl']).agg({"property_id":"nunique"}).reset_index()
test[test['property_id']>1]

,bbl,property_id
22,10001675090,2
53,10004200010,2
60,10005575010,3
70,10006975020,2
89,10012800260,3
...,...,...
8092,50059305900,4
8094,50059500150,9
8128,50398300010,2
8129,50398300450,3


In [203]:
## spot checking those 359 instances of PPls with more than one unsiue property 1i
building_energy_info_canopy[building_energy_info_canopy['bbl']=='10004200010']

,property_id,year_ending_year,weather_normalized_site_energy_use_kbtu,weather_normalized_site_energy,site_energy_use_kbtu,property_gfa_calculated_buildings_ft,property_gfa_epa_calculated,property_gfa_self_reported_ft,weather_normalized_site_eui_kbtu_ft,weather_normalized_site_eui,number_of_active_energy_meters_used_to_compute_metrics,alert_energy_no_meters_selected_for_metrics,alert_energy_meter_has_less_than_12_full_calendar_months_of_data,alert_energy_meter_has_overlaps,alert_energy_meter_has_gaps,alert_energy_meter_has_single_entry_more_than_65_days,alert_energy_no_meters_selected_for_metrics,latitude,longitude,address_line1_clean,address_line2_clean,borough_clean,city_clean,state_clean,BIN,api_matched_address,boro_code,BldgClass,Block,LandUse,NumFloors,UnitsRes,stories,ct2010,boroct2010,raw_clean_address,bbl,canopy_change_class
602,4039462.0,2017,NaN,25136390.2,24404561.5,NaN,NaN,NaN,NaN,89.9,NaN,NaN,NaN,NaN,Ok,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
7009,4039462.0,2016,NaN,23164416.9,22470279.9,NaN,NaN,NaN,NaN,82.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
12792,4039462.0,2015,NaN,24332716.1,24723102.1,NaN,279600.0,NaN,NaN,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
22563,4039462.0,2014,NaN,23872638.8,24959229.8,NaN,NaN,NaN,NaN,85.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
29207,3089482.0,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain
31947,4039462.0,2013,NaN,21185029.9,21199896.6,NaN,NaN,NaN,NaN,75.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706867,-74.00699,100 Maiden Ln,NaN,MANHATTAN,New York,NY,1001014.0,"100 MAIDEN LANE, New York, NY, USA",1,D6,42.0,04,24.0,340.0,24.0,000700,1000700,"100 Maiden Ln, MANHATTAN, NY",10004200010,Gain


In [ ]:
### Building Footprint data and Building Historic INFO and Height Ets
# BUILDING_HISTORIC
# https://data.cityofnewyork.us/City-Government/BUILDING_HISTORIC/ipkp-snf6/about_data

#Building Elevation and Subgrade (BES)
# https://data.cityofnewyork.us/City-Government/Building-Elevation-and-Subgrade-BES-/bsin-59hv/about_data